In [3]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "config")
os.environ.setdefault("DJANGO_CONFIGURATION", "Local")

import configurations
configurations.setup()

# import django
from django.conf import settings
# settings.configure(DEBUG=True)
# django.setup()


In [5]:
# Now this script or any imported module can use any part of Django it needs.
from djstruct import models

In [4]:
settings.INSTALLED_APPS

('django.contrib.admin',
 'django.contrib.auth',
 'django.contrib.contenttypes',
 'django.contrib.sessions',
 'django.contrib.messages',
 'django.contrib.staticfiles',
 'rest_framework',
 'rest_framework.authtoken',
 'django_rq',
 'versatileimagefield',
 'django_extensions',
 'authentication',
 'users',
 'djstruct')

In [6]:
import yaml

In [20]:
IMPORD_DATA_DIR = '/Users/ivan/Projects/Minireference/STRUCTURE/structure-api/data/miniref'
context_yaml_file = os.path.join(IMPORD_DATA_DIR,'context.yml')
context = yaml.load(open(context_yaml_file))

data_filenames = ['advanced_math.yml', 'cs.yml', 'math.yml', 'physics.yml']
all_data = []
for filename in data_filenames:
    this_yaml_file = os.path.join(IMPORD_DATA_DIR, filename)
    this_data = yaml.load(open(this_yaml_file))
    all_data += this_data


In [31]:
all_data[5]

{'description': "Euclidian geometry assumaes the space we're working in is flat. Flat space is all about triangles\n\n but things get different as we move to three dimensions.\n In three dimensions, the sum of angles of a triangle do not add to 180 degrees and the notion of circle is replaced by spheres. \n Bolyai and Lobachevskii laid the foundations of non-euclidean geometry that was taken further by Riemann with his Riemannian Geometry that deals with study of manifolds. \n Riemannian Geometry is a branch of Differential Geometry that uses the techniques of differential calculus and integral calculus, \n as well as linear algebra and multilinear algebra, to study problems in geometry.\n",
 'path': 'math/geometry/non-euclidean',
 'usedfors': [{'explain': u'To understand Einsten\u2019s general theory of relativity you need to know the math of curved spaces.',
   'usedfor': 'physics/general_relativity'}]}

In [46]:

class PrerequisteReference(object):
    """
    A wrapper that accepts either a string or a dict prerequisite data.
    """
    def __init__(self, data):
        if isinstance(data, basestring):
            self.prerequisite = data
            self.explain = None
            self.level = None
        elif isinstance(data, dict):
            self.prerequisite = data.get('prerequisite')
            self.explain = data.get('explain', None)
            self.level = data.get('level', None)
        else:
            raise ValueError('data must be string or dict')

class UsedforReference(object):
    """
    A wrapper that accepts either a string or a dict usedfor data.
    """
    def __init__(self, data):
        if isinstance(data, basestring):
            self.usedfor = data
            self.explain = None
            self.level = None
        elif isinstance(data, dict):
            self.usedfor = data.get('usedfor')
            self.explain = data.get('explain', None)
            self.level = data.get('level', None)
        else:
            raise ValueError('data must be string or dict')

{'explain': None, 'level': None, 'prerequisite': 'math/calc'}

In [47]:
all_paths = set()
all_path_refs = set()
for datum in all_data:
    all_paths.add(datum['path'])
    # print datum['path']
    if 'prerequisites' in datum and datum['prerequisites'] is not None:
        for prereq_str_or_dict in datum['prerequisites']:
            prereq = PrerequisteReference(prereq_str_or_dict)
            all_path_refs.add(prereq.prerequisite)
    if 'usedfors' in datum:
        for usedfor_str_or_dict in datum['usedfors']:
            usedfor = UsedforReference(usedfor_str_or_dict)
            all_path_refs.add(usedfor.usedfor)
    # TODO: related


In [44]:
all_paths

{'cs/intro',
 'math/algebra',
 'math/algebraic_geometry',
 'math/analysis',
 'math/analysis/complex',
 'math/calculus/derivatives',
 'math/calculus/integrals',
 'math/calculus/multivariable',
 'math/calculus/ofvariations',
 'math/calculus/vector',
 'math/combinatorics',
 'math/differential_equations',
 'math/differential_geometry',
 'math/functional_analysis',
 'math/geometry/non-euclidean',
 'math/linear_algebra',
 'math/logic',
 'math/measure_theory',
 'math/number_theory',
 'math/partial_differential_equations',
 'math/probability',
 'math/set_theory',
 'math/statistics',
 'math/topology',
 'physics/classical_mechanics',
 'physics/electricity',
 'physics/electricity/circuits',
 'physics/electromagnetism',
 'physics/magnetism',
 'physics/mechanics',
 'physics/mechanics/laboratory',
 'physics/optics',
 'physics/quantum/advanced',
 'physics/quantum_mechanics',
 'physics/special_relativity',
 'physics/waves'}

In [48]:
all_path_refs

{'cs/computer_graphics',
 'cs/information_theory',
 'cs/information_theory/quantum',
 'cs/machine_learning',
 'eng/electronics',
 'engineering/signals_and_systems',
 'math/abstract_algebra',
 'math/algebra',
 'math/calculus',
 'math/calculus/derivatives',
 'math/calculus/integrals',
 'math/calculus/multivariable',
 'math/calculus/multivariate',
 'math/calculus/ofvariations',
 'math/calculus/vector',
 'math/differential_equations',
 'math/functions',
 'math/geometry',
 'math/linear_algebra',
 'math/linear_algebra|GRAD',
 'math/measure_theory',
 'math/numbers',
 'math/optimization',
 'math/partial_differential_equations',
 'math/probability',
 'math/probability|GRAD',
 'math/statistics',
 'physics/classical_mechanics',
 'physics/electricity',
 'physics/electromagnetism|GRAD',
 'physics/general_relativity',
 'physics/hamiltonian_mechanics',
 'physics/magnetism',
 'physics/mechanics',
 'physics/mechanics/laboratory',
 'physics/modern_physics',
 'physics/particle_physics',
 'physics/quantum

In [49]:
all_path_refs - all_paths

{'cs/computer_graphics',
 'cs/information_theory',
 'cs/information_theory/quantum',
 'cs/machine_learning',
 'eng/electronics',
 'engineering/signals_and_systems',
 'math/abstract_algebra',
 'math/calculus',
 'math/calculus/multivariate',
 'math/functions',
 'math/geometry',
 'math/linear_algebra|GRAD',
 'math/numbers',
 'math/optimization',
 'math/probability|GRAD',
 'physics/electromagnetism|GRAD',
 'physics/general_relativity',
 'physics/hamiltonian_mechanics',
 'physics/modern_physics',
 'physics/particle_physics',
 'physics/quantum/theory',
 'physics/quantum_physics',
 'physics/solid_state_physics',
 'physics/thermodynamics'}